In [1]:
import pandas as pd
import numpy as np
import xlrd
import re

In [2]:
table_id = "236.30"
table_year = "2019"
file = "tabn236.30.xls"

## Create Table Info

In [ ]:
def skiplines(sheet):
    
    # determine title rows to skip
    if re.match(r"\[.*\]", sheet.cell_value(1,0)):
        skip = 2
    else:
        skip = 1
        
    return skip

In [63]:
def create_table_info(digest_table_id, digest_table_year, table_file, sh, df):

    ## Table Title
    prog = re.compile(r"Table (\d{3}\.\d{2})\. (.*)")
    result = prog.match(sh.cell_value(0,0))
    table_title = result.group(2)
    
    skip = skiplines(sh)

    # headnote
    headnote = sh.cell_value(1,0)

    # stub_head
    stub_head = sh.cell_value(2,0)

    # general_note
    general = df[0].str.extract(r"NOTE: (.*)").dropna()
    general_note = general[0].values[0].strip()

    # source
    source = df[0].str.extract(r"SOURCE: (.*)\((.*)\)").dropna()
    source_note = source[0].values[0].strip()

    # last_prepared
    last_prepared = source[1].values[0].strip()

    col_list = [
        'digest_table_id', 
        'digest_table_year', 
        'table_title',
        'headnote',
        'stub_head',
        'general_note',
        'source_note',
        'last_prepared'
    ]

    val_list = [
        digest_table_id, 
        digest_table_year, 
        table_title,
        headnote,
        stub_head,
        general_note,
        source_note,
        last_prepared
    ]

    tb_info = pd.DataFrame(np.array([col_list, val_list]))
    return tb_info

## Create Column Info Sheet

In [4]:
def AA(num, string):
    """Recursively builds column index
    
    Inspired by from this Stackoverflow answer:
    https://stackoverflow.com/a/54837286
    """
    
    r = num % 26
    num = (num - r) // 26
    string = chr(ord("A") + r) + string
    
    if num > 26:
        string = AA(num, string)
    elif num > 0:
        string = chr(ord("A") + num - 1) + string
        
    return string

In [37]:
def header_end(sheet, df):
    """Returns the row number of the integer row"""
    
    for row in range(0,sheet.nrows):
        if sheet.cell_value(row,0) == 1:
            return row

# code to check full row
#     for row in range(0,sheet.nrows):
#         if list(df.iloc[row,:]) == list(range(1,sheet.ncols+1)):
#             return row
        
    print("End of file reached, no integer row")
    return 0

In [52]:
def attach_header(file_name, sheet, df):
    """Returns the same dataframe with formatted column headers"""
    
    header_n = header_end(sheet, df)
    
    # determine title rows to skip
    if re.match(r"\[.*\]", sheet.cell_value(1,0)):
        skip = 2
    else:
        skip = 1
    
    header = pd.read_excel(file_name, 
                               skiprows=skip, 
                               header=None, 
                               nrows=header_n-skip,
                               usecols=list(range(1,sheet.ncols))
                          )
    header = header.ffill(axis=0).ffill(axis=1)
    data = pd.read_excel(file_name,
                         skiprows=header_n + 1,
                         header=None,
                         usecols=list(range(1,sheet.ncols))
                        )
    data.columns = pd.MultiIndex.from_arrays(header.values)
    
    return data

In [54]:
def create_col_info(df):
    """Returns DataFrame with column information"""
    
    # convert header df to col_info dataframe
    col_info = df.columns.to_frame(index=False)
    is_duplicate = col_info.apply(lambda row: row.duplicated(), axis=1)
    col_info = col_info.where(~is_duplicate, "")
    
    # create extra columns for unused columns index levels
    for x in range(col_info.shape[1], 7):
        col_info.insert(x, x, "")
    
    # label column levels
    col_info.columns = [f"column_level_{col+1}" for col in col_info.columns]
    
    # add table_id and table_year to col_info
    col_info["digest_table_id"] = table_id
    col_info["digest_table_year"] = table_year
    
    # create column_index field
    col_info["column_index"] = [AA(i,"") for i in col_info.index]
    
    return col_info

In [55]:
def add_footnotes(df, col_info):
    """Adds footnote columns and sorts columns"""
    
    # Extract footnotes from raw df
    footnotes = df[0].str.extract(r"\\([0-9])\\(.*)").dropna()
    footnotes.columns = ["number", "note"]
    footnotes = footnotes.set_index("number")

    # Extract footnotes from raw df
    footnotes = df[0].str.extract(r"\\([0-9])\\(.*)").dropna().set_index(0)
    footnotes_dict = footnotes.to_dict()[1]

    # create column_ref_note columns
    for x in range(1,8):
        col = col_info[f"column_level_{x}"]

        # create a reference column with the footnote number
        refs = col.str.extract(r"\\([0-9])\\")

        # create new column with the reference note
        col_info[f"column_ref_note_{x}"] = refs.replace(footnotes_dict)

        # delete footnote from column_level_x
        col_level = col.str.replace(
            pat = r"\\[0-9]\\",
            repl = ""
        )

        col_info[f"column_level_{x}"] = col_level

    # Remove extra headers
    col_info = col_info.fillna("")

    # list of columns in the desired order
    col_list = [[f"column_level_{x}", f"column_ref_note_{x}"] for x in range(1,8)]
    col_list = list(np.array(col_list).flatten())

    # rearrange column order
    col_info = col_info[
        ['digest_table_id', 'digest_table_year', 'column_index'] + 
        col_list
    ]
    
    return col_info

### Output to Excel

In [64]:
def generate_table(digest_table_id, digest_table_year, table_file):
    """Parses table and generates output"""

    book = xlrd.open_workbook(table_file,formatting_info=True)
    sh = book.sheet_by_index(0)
    font = book.font_list

    # read in the raw dataset from excel
    raw_df = pd.read_excel(table_file, header=None)

    # create output filename
    digest_number = digest_table_id.replace(".", "_")
    output_file = f"{digest_table_year}_{digest_number}_activate_step1.xlsx"

    table_info = create_table_info(digest_table_id, 
                                   digest_table_year, 
                                   table_file, 
                                   sh, 
                                   raw_df
                                  )

    data = attach_header(table_file, sh, raw_df)
    col_info = create_col_info(data)
    col_info = add_footnotes(raw_df, col_info)

    # make column names part of dataframe
    col_info = pd.DataFrame(np.vstack([col_info.columns, col_info]))
    
    return book, sh, font, raw_df, output_file, data, table_info, col_info

In [10]:
def write_xls(output_file, table_info, col_info):
    """Writes to output file"""

    with pd.ExcelWriter(output_file) as writer:
        table_info.to_excel(
            writer, 
            sheet_name="table_info", 
            index=False,
            header=False
        )
        col_info.to_excel(
            writer, 
            sheet_name="column_info",
            index=False,
            header=False
        )

In [65]:
book, sh, font, raw_df, out_file, data, tab, cols = generate_table(table_id, table_year, file)

In [66]:
book1, sh1, font1, raw_df1, out_file1, data1, tab1, cols1 = generate_table("203.50", "2019", "tabn203.50.xls")

In [67]:
tab

,0,1,2,3,4,5,6,7
0,digest_table_id,digest_table_year,table_title,headnote,stub_head,general_note,source_note,last_prepared
1,236.30,2019,Total expenditures for public elementary and s...,[In thousands of current dollars],State or jurisdiction,Excludes expenditures for state education agen...,"U.S. Department of Education, National Center ...",This table was prepared August 2019.


In [73]:
cols

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,digest_table_id,digest_table_year,column_index,column_level_1,column_ref_note_1,column_level_2,column_ref_note_2,column_level_3,column_ref_note_3,column_level_4,column_ref_note_4,column_level_5,column_ref_note_5,column_level_6,column_ref_note_6,column_level_7,column_ref_note_7
1,236.30,2019,A,Total expenditures,,Total,,,,,,,,,,,
2,236.30,2019,B,Total expenditures,,Current expenditures for elementary and second...,,"Elementary/ secondary current expenditures, total",,,,,,,,,
3,236.30,2019,C,Total expenditures,,Current expenditures for elementary and second...,,Instruction,,,,,,,,,
4,236.30,2019,D,Total expenditures,,Current expenditures for elementary and second...,,Support services,,"Support services, total",,,,,,,
5,236.30,2019,E,Total expenditures,,Current expenditures for elementary and second...,,Support services,,Student support,"Includes expenditures for guidance, health, at...",,,,,,
6,236.30,2019,F,Total expenditures,,Current expenditures for elementary and second...,,Support services,,Instruc- tional staff,Includes expenditures for curriculum developme...,,,,,,
7,236.30,2019,G,Total expenditures,,Current expenditures for elementary and second...,,Support services,,General adminis- tration,,,,,,,
8,236.30,2019,H,Total expenditures,,Current expenditures for elementary and second...,,Support services,,School adminis- tration,,,,,,,
9,236.30,2019,I,Total expenditures,,Current expenditures for elementary and second...,,Support services,,Operation and maintenance,,,,,,,


In [68]:
tab1

,0,1,2,3,4,5,6,7
0,digest_table_id,digest_table_year,table_title,headnote,stub_head,general_note,source_note,last_prepared
1,203.50,2019,Enrollment and percentage distribution of enro...,Region and year,,Race categories exclude persons of Hispanic et...,"U.S. Department of Education, National Center ...",This table was prepared December 2019.


In [69]:
cols1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,digest_table_id,digest_table_year,column_index,column_level_1,column_ref_note_1,column_level_2,column_ref_note_2,column_level_3,column_ref_note_3,column_level_4,column_ref_note_4,column_level_5,column_ref_note_5,column_level_6,column_ref_note_6,column_level_7,column_ref_note_7
1,236.30,2019,A,Enrollment (in thousands),,Total,,,,,,,,,,,
2,236.30,2019,B,Enrollment (in thousands),,White,,,,,,,,,,,
3,236.30,2019,C,Enrollment (in thousands),,Black,,,,,,,,,,,
4,236.30,2019,D,Enrollment (in thousands),,His- panic,,,,,,,,,,,
5,236.30,2019,E,Enrollment (in thousands),,Asian,,,,,,,,,,,
6,236.30,2019,F,Enrollment (in thousands),,Asian,,,,,,,,,,,
7,236.30,2019,G,Enrollment (in thousands),,Pacific Islander,,,,,,,,,,,
8,236.30,2019,H,Enrollment (in thousands),,American Indian/\nAlaska\nNative,,,,,,,,,,,
9,236.30,2019,I,Enrollment (in thousands),,Two or more races,,,,,,,,,,,


In [71]:
data1

Enrollment (in thousands)                                                  \
                       Total      White     Black His- panic     Asian Asian   
0                        NaN        NaN       NaN        NaN       NaN   NaN   
1                  44840.481  29044.127  7550.570   6072.145  1668.439   \1\   
2                  47203.539  28877.548  8099.788   7725.843  1950.330   \1\   
3                  47671.843  28734.542  8176.918   8169.185  2027.612   \1\   
4                  48183.086  28618.399  8298.857   8594.172  2088.478   \1\   
..                       ...        ...       ...        ...       ...   ...   
71                       NaN        NaN       NaN        NaN       NaN   NaN   
72                       NaN        NaN       NaN        NaN       NaN   NaN   
73                       NaN        NaN       NaN        NaN       NaN   NaN   
74                       NaN        NaN       NaN        NaN       NaN   NaN   
75                       NaN        NaN       NaN        NaN       NaN   NaN   

                                                                        \
   Pacific Islander American Indian/\nAlaska\nNative Two or more races   
0               NaN                              NaN               NaN   
1               ---                          505.200               ---   
2               ---                          550.030               ---   
3               ---                          563.586               ---   
4               ---                          583.180               ---   
..              ...                              ...               ...   
71              NaN                              NaN               NaN   
72              NaN                              NaN               NaN   
73              NaN                              NaN               NaN   
74              NaN                              NaN               NaN   
75              NaN                              NaN               NaN   

                     Percentage distribution                                   \
   Two or more races                   Total      White      Black His- panic   
0                NaN                     NaN        NaN        NaN        NaN   
1                NaN                   100.0  64.772113  16.838736  13.541659   
2                NaN                   100.0  61.176659  17.159281  16.367084   
3                NaN                   100.0  60.275710  17.152511  17.136289   
4                NaN                   100.0  59.395114  17.223590  17.836491   
..               ...                     ...        ...        ...        ...   
71               NaN                     NaN        NaN        NaN        NaN   
72               NaN                     NaN        NaN        NaN        NaN   
73               NaN                     NaN        NaN        NaN        NaN   
74               NaN                     NaN        NaN        NaN        NaN   
75               NaN                     NaN        NaN        NaN        NaN   

                                                                      \
       Asian Asian Pacific Islander American Indian/\nAlaska\nNative   
0        NaN   NaN              NaN                              NaN   
1   3.720832   \1\              ---                         1.126661   
2   4.131745   \1\              ---                         1.165230   
3   4.253270   \1\              ---                         1.182220   
4   4.334463   \1\              ---                         1.210342   
..       ...   ...              ...                              ...   
71       NaN   NaN              NaN                              NaN   
72       NaN   NaN              NaN                              NaN   
73       NaN   NaN              NaN                              NaN   
74       NaN   NaN              NaN                              NaN   
75       NaN   NaN              NaN                              NaN   

        

In [12]:
write_xls(out_file, tab, cols)

In [72]:
write_xls(out_file1, tab1, cols1)

## Row Info Tab

In [13]:
sh.cell_value(0,0)

'Table 236.30. Total expenditures for public elementary and secondary education and other related programs, by function and state or jurisdiction: 2016-17'

In [14]:
sh.cell_value(7,0)

'   United States ........'

In [15]:
cell_xf = book.xf_list[sh.cell_xf_index(7,0)]
font[cell_xf.font_index].bold

1

In [16]:
sh.cell_value(13,0)

''

In [17]:
re.search("\S", sh.cell_value(13,0)) if re.search("\S", sh.cell_value(13,0)) else 0

0

In [18]:
row_info = data.copy()

In [19]:
# create is_total
start = 7
is_total = []

for row in range(start,sh.nrows):
    cell_val = sh.cell_value(row,0)
    cell_xf = book.xf_list[sh.cell_xf_index(row,0)]
    
    is_bold = font[cell_xf.font_index].bold
    
    result = re.search("\S", cell_val)
    indent_n = result.start() if result else 0
    
    is_total.append(bool(is_bold) and (indent_n in [3,5]))

In [20]:
row_info["is_total"] = is_total

In [21]:
# create row_index
# should be created after NaN rows have been dropped
row_info["row_index"] = row_info.index + 1

In [22]:
row_info.head(10)

Total expenditures    \
                 Total   
                 Total   
                 Total   
0         7.076013e+08   
1         8.030225e+06   
2         2.582582e+06   
3         1.053083e+07   
4         5.622673e+06   
5         8.796822e+07   
6                  NaN   
7         1.063274e+07   
8         1.157366e+07   
9         2.247039e+06   

                                                                              \
  Current expenditures for elementary and secondary programs                   
             Elementary/ secondary current expenditures, total   Instruction   
             Elementary/ secondary current expenditures, total   Instruction   
0                                       6.191646e+08            3.760695e+08   
1                                       7.097472e+06            4.049192e+06   
2                                       2.367707e+06            1.266042e+06   
3                                       8.966684e+06            4.828965e+06   
4                                       4.936465e+06            2.769224e+06   
5                                       7.666373e+07            4.544206e+07   
6                                                NaN                     NaN   
7                                       8.913931e+06            4.989814e+06   
8                                       1.066457e+07            6.722928e+06   
9                                       2.029229e+06            1.269553e+06   

                                                                       \
                                                                        
         Support services                                               
  Support services, total Student support\4\ Instruc- tional staff\5\   
0            2.177547e+08       3.594604e+07             2.916935e+07   
1            2.563159e+06       4.461755e+05             2.985193e+05   
2            1.013894e+06       1.836071e+05             1.965253e+05   
3            3.669286e+06       6.876655e+05             4.345531e+05   
4            1.898530e+06       2.669395e+05             4.151244e+05   
5            2.815535e+07       4.598429e+06             4.880880e+06   
6                     NaN                NaN                      NaN   
7            3.570698e+06       5.024576e+05             5.131430e+05   
8            3.614923e+06       6.847691e+05             3.354516e+05   
9            6.919073e+05       9.064999e+04             3.724782e+04   

                                                                              \
                                                                               
                                                                               
  General adminis- tration School adminis- tration Operation and maintenance   
0             1.230519e+07            3.456894e+07              5.743347e+07   
1             1.824433e+05            4.427333e+05              6.600538e+05   
2             3.385600e+04            1.447664e+05              2.838423e+05   
3             1.739409e+05            5.029053e+05              1.098094e+06   
4             1.248428e+05            2.588572e+05              5.004338e+05   
5             7.537747e+05            5.082339e+06              7.573197e+06   
6                      NaN                     NaN                       NaN   
7             1.445380e+05            6.792786e+05              8.145828e+05   
8             2.392059e+05            6.241178e+05              9.156950e+05   
9             3.189485e+04            1.286915e+05              2.166755e+05   

                                                                 \
                                                                  
                                                  Food services   
  Student transpor- tation Other support services Food services   
0             2.535029e+07           2.298140e+07  2.409522e+07   
1             3.666710e+05           

In [17]:
import scapetables

In [18]:
scapetables.skiplines(sh)

NameError: name 'sh' is not defined